In [ ]:
# Import the unified training data system
from unifiedTrainingData import (
    createUnifiedTrainingData, 
    loadUnifiedTrainingData, 
    getUnifiedTrainingDataStats,
    prepareUnifiedTrainingDataForModel
)

# Define your keywords
keywords = [
    'tesla', 'bmw', 'mercedes', 'audi', 'volkswagen', 'toyota', 'honda', 'ford', 
    'chevrolet', 'hyundai', 'kia', 'nissan', 'volvo', 'lexus', 'porsche', 
    'electric car', 'ev', 'electric vehicle', 'hybrid car', 'autonomous car',
    'self driving car', 'car sharing', 'ride sharing', 'uber', 'lyft',
    'car rental', 'car insurance', 'car maintenance', 'car repair',
    'car dealership', 'car financing', 'car loan', 'car lease'
]

# Create unified training data
summary = createUnifiedTrainingData(
    keywords=keywords,
    startDate="2025-06-01",
    endDate="2025-06-30",
    model='text-embedding-ada-002',
    filename='data/unified_training_data.json'
)

print(f"\n✅ Training data creation completed!")
print(f"Success rate: {summary['success_rate']:.2%}")

In [ ]:
# Load and verify the created training data
print("\nVerifying created training data...")
data = loadUnifiedTrainingData()

print(f"Total keywords in training data: {len(data)}")

# Show sample data structure
if data:
    sample_keyword = list(data.keys())[0]
    embedding, chatgpt_trend, google_trend = data[sample_keyword]
    print(f"\nSample data for '{sample_keyword}':")
    print(f"  Embedding shape: {embedding.shape}")
    print(f"  ChatGPT trend shape: {chatgpt_trend.shape}")
    print(f"  Google trend shape: {google_trend.shape}")
    print(f"  Google trend sample: {google_trend[:5]}")
    print(f"  ChatGPT trend sample: {chatgpt_trend[:5]}")

# Prepare data for the embedding KNN model
print(f"\nPreparing data for embedding KNN model...")
prepareUnifiedTrainingDataForModel(data)
print(f"✅ Data prepared for model use!")

# Show final statistics
stats = getUnifiedTrainingDataStats()
print(f"\nFinal training data statistics:")
print(f"  Total keywords: {stats['total_keywords']}")
print(f"  File size: {stats['file_size_mb']:.2f} MB")
print(f"  Keywords: {stats['keywords'][:10]}...")  # Show first 10 keywords

In [ ]:
# Test inference with the trained model
from embeddingsKNN import quickKeywordAnalysis, analyzeNewKeyword

# Test keywords
test_keywords = ['tesla model 3', 'bmw i4', 'hyundai ioniq']

print("Testing inference with trained model...")
print("="*50)

for keyword in test_keywords:
    print(f"\nTesting: {keyword}")
    try:
        # Quick analysis
        result = quickKeywordAnalysis(keyword, "2025-06-01", "2025-06-30")
        print(f"  Google trend mean: {result['google_trend_mean']:.1f}")
        print(f"  Predicted ChatGPT mean: {result['predicted_chatgpt_mean']:.1f}")
        print(f"  Prediction confidence: {result['prediction_confidence']:.3f}")
        print(f"  Nearest neighbors: {result['nearest_neighbors']}")
        
    except Exception as e:
        print(f"  ❌ Error: {e}")

In [ ]:
# Import required libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from embeddingsKNN import quickKeywordAnalysis, analyzeNewKeyword
from unifiedTrainingData import loadUnifiedTrainingData, prepareUnifiedTrainingDataForModel

# Load and prepare the training data for the model
print("Loading unified training data...")
data = loadUnifiedTrainingData()
prepareUnifiedTrainingDataForModel(data)
print(f"✅ Model prepared with {len(data)} keywords")

# Test keywords for plotting
test_keywords = ['tesla model 3', 'bmw i4', 'hyundai ioniq', 'electric suv', 'autonomous driving']

# Create comprehensive plots
fig, axes = plt.subplots(len(test_keywords), 2, figsize=(15, 5*len(test_keywords)))
if len(test_keywords) == 1:
    axes = axes.reshape(1, -1)

print(f"Analyzing and plotting results for {len(test_keywords)} keywords...")

for i, keyword in enumerate(test_keywords):
    print(f"Processing {i+1}/{len(test_keywords)}: {keyword}")
    
    try:
        # Get detailed analysis
        result = analyzeNewKeyword(keyword, "2025-06-01", "2025-06-30")
        
        # Extract data
        google_trends = result['google_trends']['trend_values']
        predicted_chatgpt = result['predicted_chatgpt_trend']['trend_values']
        actual_chatgpt = result['actual_chatgpt_trend']['trend_values']
        dates = pd.date_range("2025-06-01", "2025-06-30", freq='D')
        
        # Plot 1: Google Trends vs Predicted ChatGPT Trends
        ax1 = axes[i, 0]
        ax1.plot(dates, google_trends, 'b-', label='Google Trends', linewidth=2, alpha=0.8)
        ax1.plot(dates, predicted_chatgpt, 'r--', label='Predicted ChatGPT', linewidth=2, alpha=0.8)
        ax1.plot(dates, actual_chatgpt, 'g-', label='Actual ChatGPT', linewidth=2, alpha=0.8)
        ax1.set_title(f'Trend Comparison: {keyword}', fontsize=14, fontweight='bold')
        ax1.set_ylabel('Trend Value', fontsize=12)
        ax1.legend(fontsize=10)
        ax1.grid(True, alpha=0.3)
        ax1.tick_params(axis='x', rotation=45)
        
        # Add correlation info
        correlation = np.corrcoef(google_trends, predicted_chatgpt)[0, 1]
        ax1.text(0.02, 0.98, f'Correlation: {correlation:.3f}', 
                transform=ax1.transAxes, fontsize=10, 
                verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
        
        # Plot 2: Neighbor Similarities
        ax2 = axes[i, 1]
        neighbors = result['neighbors']
        neighbor_names = [n['keyword'] for n in neighbors]
        similarities = [n['similarity'] for n in neighbors]
        
        bars = ax2.bar(neighbor_names, similarities, color='skyblue', alpha=0.7)
        ax2.set_title(f'Nearest Neighbors: {keyword}', fontsize=14, fontweight='bold')
        ax2.set_ylabel('Similarity Score', fontsize=12)
        ax2.set_ylim(0, 1)
        
        # Add similarity values on bars
        for bar, similarity in zip(bars, similarities):
            ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                    f'{similarity:.3f}', ha='center', va='bottom', fontsize=9)
        
        ax2.tick_params(axis='x', rotation=45)
        ax2.grid(True, alpha=0.3)
        
        # Add prediction confidence
        confidence = result['analysis_summary']['prediction_confidence']
        ax2.text(0.02, 0.98, f'Confidence: {confidence:.3f}', 
                transform=ax2.transAxes, fontsize=10, 
                verticalalignment='top', bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.8))
        
    except Exception as e:
        print(f"❌ Error processing '{keyword}': {e}")
        # Create empty plots for failed keywords
        axes[i, 0].text(0.5, 0.5, f'Error: {str(e)[:50]}...', 
                       ha='center', va='center', transform=axes[i, 0].transAxes,
                       bbox=dict(boxstyle='round', facecolor='lightcoral', alpha=0.8))
        axes[i, 0].set_title(f'Error: {keyword}', fontsize=14, fontweight='bold')
        axes[i, 1].text(0.5, 0.5, 'No data available', 
                       ha='center', va='center', transform=axes[i, 1].transAxes)
        axes[i, 1].set_title(f'No neighbors: {keyword}', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

# Summary statistics
print(f"\n{'='*60}")
print("SUMMARY STATISTICS")
print(f"{'='*60}")

summary_data = []
for keyword in test_keywords:
    try:
        result = quickKeywordAnalysis(keyword, "2025-06-01", "2025-06-30")
        summary_data.append({
            'Keyword': keyword,
            'Google Trend Mean': f"{result['google_trend_mean']:.1f}",
            'Predicted ChatGPT Mean': f"{result['predicted_chatgpt_mean']:.1f}",
            'Prediction Confidence': f"{result['prediction_confidence']:.3f}",
            'Trend Magnitude': result['trend_magnitude'],
            'Volatility Level': result['volatility_level'],
            'Nearest Neighbors': ', '.join(result['nearest_neighbors'][:3])
        })
    except Exception as e:
        summary_data.append({
            'Keyword': keyword,
            'Google Trend Mean': 'Error',
            'Predicted ChatGPT Mean': 'Error',
            'Prediction Confidence': 'Error',
            'Trend Magnitude': 'Error',
            'Volatility Level': 'Error',
            'Nearest Neighbors': 'Error'
        })

# Create summary table
summary_df = pd.DataFrame(summary_data)
print(summary_df.to_string(index=False))

# Additional detailed analysis plot
print(f"\n{'='*60}")
print("DETAILED ANALYSIS PLOT")
print(f"{'='*60}")

# Create a single comprehensive plot for one keyword
if test_keywords:
    keyword = test_keywords[0]  # Use first keyword for detailed analysis
    try:
        result = analyzeNewKeyword(keyword, "2025-06-01", "2025-06-30")
        
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
        
        dates = pd.date_range("2025-06-01", "2025-06-30", freq='D')
        google_trends = result['google_trends']['trend_values']
        predicted_chatgpt = result['predicted_chatgpt_trend']['trend_values']
        actual_chatgpt = result['actual_chatgpt_trend']['trend_values']
        
        # Plot 1: All trends together
        ax1.plot(dates, google_trends, 'b-', label='Google Trends', linewidth=2)
        ax1.plot(dates, predicted_chatgpt, 'r--', label='Predicted ChatGPT', linewidth=2)
        ax1.plot(dates, actual_chatgpt, 'g-', label='Actual ChatGPT', linewidth=2)
        ax1.set_title(f'Complete Trend Analysis: {keyword}', fontsize=14, fontweight='bold')
        ax1.set_ylabel('Trend Value')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        ax1.tick_params(axis='x', rotation=45)
        
        # Plot 2: Prediction accuracy
        ax2.scatter(actual_chatgpt, predicted_chatgpt, alpha=0.6, color='purple')
        ax2.plot([0, 100], [0, 100], 'r--', alpha=0.5, label='Perfect Prediction')
        ax2.set_xlabel('Actual ChatGPT Trend')
        ax2.set_ylabel('Predicted ChatGPT Trend')
        ax2.set_title('Prediction Accuracy', fontsize=14, fontweight='bold')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        # Plot 3: Trend correlation
        ax3.plot(dates, google_trends, 'b-', label='Google Trends', alpha=0.7)
        ax3.plot(dates, predicted_chatgpt, 'r-', label='Predicted ChatGPT', alpha=0.7)
        ax3.set_xlabel('Date')
        ax3.set_ylabel('Trend Value')
        ax3.set_title('Google vs Predicted ChatGPT Correlation', fontsize=14, fontweight='bold')
        ax3.legend()
        ax3.grid(True, alpha=0.3)
        ax3.tick_params(axis='x', rotation=45)
        
        # Plot 4: Neighbor similarities
        neighbors = result['neighbors']
        neighbor_names = [n['keyword'] for n in neighbors]
        similarities = [n['similarity'] for n in neighbors]
        
        bars = ax4.bar(neighbor_names, similarities, color='orange', alpha=0.7)
        ax4.set_title('Nearest Neighbors Similarity', fontsize=14, fontweight='bold')
        ax4.set_ylabel('Similarity Score')
        ax4.set_ylim(0, 1)
        ax4.tick_params(axis='x', rotation=45)
        
        # Add values on bars
        for bar, similarity in zip(bars, similarities):
            ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                    f'{similarity:.3f}', ha='center', va='bottom')
        
        plt.tight_layout()
        plt.show()
        
        # Print detailed metrics
        print(f"\nDetailed Metrics for '{keyword}':")
        print(f"  MSE: {result['prediction_metrics']['mse']:.4f}")
        print(f"  MAE: {result['prediction_metrics']['mae']:.4f}")
        print(f"  RMSE: {result['prediction_metrics']['rmse']:.4f}")
        print(f"  Correlation: {result['analysis_summary']['trend_correlation']:.4f}")
        print(f"  Confidence: {result['analysis_summary']['prediction_confidence']:.4f}")
        
    except Exception as e:
        print(f"❌ Error in detailed analysis for '{keyword}': {e}")

print(f"\n{'='*60}")
print("ANALYSIS COMPLETED")
print(f"{'='*60}")

# Prompt Inference

In [ ]:
# Import required functions
from embeddingsKNN import promptToKeywordsAndAnalyze, loadTrainingData
import matplotlib.pyplot as plt
import numpy as np

# Load training data
print("🔄 Loading training data...")
loadTrainingData()
print("✅ Training data loaded successfully")

# Run inference on the prompt
print("\n" + "="*70)
print("🚀 RUNNING INFERENCE ON PROMPT: 'is the model 3 a good car?'")
print("="*70)

# Execute the analysis
result = promptToKeywordsAndAnalyze(
    prompt="is the model 3 a good car?",
    startDate="2025-06-01",
    endDate="2025-06-30",
    k=3,
    weights='distance'
)

print(f"\n Analysis completed!")
print(f"📝 Original prompt: '{result['prompt']}'")
print(f" Generated keywords: {result['generated_keywords']}")
print(f"✅ Successfully analyzed: {result['successful_keywords']}")
print(f" Keywords processed: {result['average_result']['num_keywords']}")

In [ ]:
# Display detailed results
print("\n" + "="*70)
print("📈 DETAILED ANALYSIS RESULTS")
print("="*70)

# Show individual keyword performance
for i, keyword in enumerate(result['successful_keywords']):
    individual = result['individual_results'][i]
    print(f"\n🔍 Keyword {i+1}: '{keyword}'")
    print(f"   Google Trend Mean: {individual['google_trends']['statistics']['mean_trend']:.1f}")
    print(f"   🤖 Predicted ChatGPT Mean: {individual['predicted_chatgpt_trend']['statistics']['mean_predicted']:.1f}")
    print(f"   🎯 Prediction Confidence: {individual['analysis_summary']['prediction_confidence']:.3f}")
    print(f"   Trend Correlation: {individual['analysis_summary']['trend_correlation']:.3f}")
    print(f"   👥 Similar Keywords: {[n['keyword'] for n in individual['neighbors']]}")

# Show aggregated results
avg = result['average_result']
print(f"\n AGGREGATED RESULTS (Average of {avg['num_keywords']} keywords):")
print(f"   📈 Average Google Trend: {avg['average_google_trend']:.2f} ± {avg['std_google_trend']:.2f}")
print(f"   🤖 Average Predicted ChatGPT: {avg['average_predicted_chatgpt']:.2f} ± {avg['std_predicted_chatgpt']:.2f}")
print(f"   Average Confidence: {avg['average_confidence']:.3f} ± {avg['std_confidence']:.3f}")
print(f"   📊 Average Correlation: {avg['average_correlation']:.3f} ± {avg['std_correlation']:.3f}")

In [ ]:
# Create additional custom visualizations
print("\n" + "="*70)
print("📊 CUSTOM VISUALIZATION ANALYSIS")
print("="*70)

# Extract data for custom plots
keywords = result['successful_keywords']
individual_results = result['individual_results']

# Create a comprehensive analysis plot
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(18, 14))

# Plot 1: Performance comparison
google_means = [r['google_trends']['statistics']['mean_trend'] for r in individual_results]
chatgpt_means = [r['predicted_chatgpt_trend']['statistics']['mean_predicted'] for r in individual_results]

x = np.arange(len(keywords))
width = 0.35

bars1 = ax1.bar(x - width/2, google_means, width, label='Google Trends', alpha=0.8, color='#1f77b4')
bars2 = ax1.bar(x + width/2, chatgpt_means, width, label='Predicted ChatGPT', alpha=0.8, color='#ff7f0e')

ax1.set_xlabel('Keywords', fontsize=12)
ax1.set_ylabel('Average Trend Value', fontsize=12)
ax1.set_title('Model 3 Analysis: Keyword Performance Comparison', fontsize=14, fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels(keywords, rotation=45, ha='right')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Add values on bars
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + 1,
                f'{height:.1f}', ha='center', va='bottom', fontsize=9, fontweight='bold')

# Plot 2: Confidence analysis
confidences = [r['analysis_summary']['prediction_confidence'] for r in individual_results]
bars3 = ax2.bar(keywords, confidences, color='#2ca02c', alpha=0.7)
ax2.set_xlabel('Keywords', fontsize=12)
ax2.set_ylabel('Prediction Confidence', fontsize=12)
ax2.set_title('Prediction Confidence by Keyword', fontsize=14, fontweight='bold')
ax2.set_ylim(0, 1)
ax2.tick_params(axis='x', rotation=45)
ax2.grid(True, alpha=0.3)

# Add confidence values on bars
for bar, conf in zip(bars3, confidences):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
            f'{conf:.3f}', ha='center', va='bottom', fontsize=9, fontweight='bold')

# Plot 3: Correlation analysis
correlations = [r['analysis_summary']['trend_correlation'] for r in individual_results]
bars4 = ax3.bar(keywords, correlations, color='#d62728', alpha=0.7)
ax3.set_xlabel('Keywords', fontsize=12)
ax3.set_ylabel('Trend Correlation', fontsize=12)
ax3.set_title('Google vs ChatGPT Trend Correlation', fontsize=14, fontweight='bold')
ax3.set_ylim(-1, 1)
ax3.tick_params(axis='x', rotation=45)
ax3.grid(True, alpha=0.3)

# Add correlation values on bars
for bar, corr in zip(bars4, correlations):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
            f'{corr:.3f}', ha='center', va='bottom', fontsize=9, fontweight='bold')

# Plot 4: Summary metrics
metrics = ['Google Trends', 'Predicted ChatGPT', 'Confidence', 'Correlation']
avg_values = [
    avg['average_google_trend'],
    avg['average_predicted_chatgpt'],
    avg['average_confidence'],
    avg['average_correlation']
]
std_values = [
    avg['std_google_trend'],
    avg['std_predicted_chatgpt'],
    avg['std_confidence'],
    avg['std_correlation']
]

colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
bars5 = ax4.bar(metrics, avg_values, yerr=std_values, capsize=5, 
               color=colors, alpha=0.7)
ax4.set_ylabel('Average Value', fontsize=12)
ax4.set_title('Overall Performance Summary', fontsize=14, fontweight='bold')
ax4.grid(True, alpha=0.3)

# Add average values on bars
for bar, avg_val in zip(bars5, avg_values):
    ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
            f'{avg_val:.3f}', ha='center', va='bottom', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Display final summary and insights
print("\n" + "="*70)
print("🎯 FINAL ANALYSIS SUMMARY")
print("="*70)

print(f"📝 Original Prompt: '{result['prompt']}'")
print(f" Generated Keywords: {', '.join(result['generated_keywords'])}")
print(f"✅ Successfully Analyzed: {', '.join(result['successful_keywords'])}")
print(f"📊 Analysis Period: June 2025")
print(f" Keywords: {avg['num_keywords']}")

print(f"\n📈 PERFORMANCE METRICS:")
print(f"   📊 Average Google Trend: {avg['average_google_trend']:.2f} ± {avg['std_google_trend']:.2f}")
print(f"   🤖 Average Predicted ChatGPT: {avg['average_predicted_chatgpt']:.2f} ± {avg['std_predicted_chatgpt']:.2f}")
print(f"   Average Confidence: {avg['average_confidence']:.3f} ± {avg['std_confidence']:.3f}")
print(f"   📊 Average Correlation: {avg['average_correlation']:.3f} ± {avg['std_correlation']:.3f}")

# Provide insights based on the results
print(f"\n KEY INSIGHTS:")
if avg['average_google_trend'] > 50:
    print(f"   High Google search interest for Model 3 related topics")
else:
    print(f"   📉 Moderate Google search interest for Model 3 related topics")

if avg['average_predicted_chatgpt'] > avg['average_google_trend']:
    print(f"   🤖 ChatGPT interest predicted to be higher than Google searches")
else:
    print(f"   📊 ChatGPT interest predicted to be lower than Google searches")

if avg['average_confidence'] > 0.7:
    print(f"   High confidence in predictions")
elif avg['average_confidence'] > 0.5:
    print(f"   ⚖️ Moderate confidence in predictions")
else:
    print(f"   ⚠️ Low confidence in predictions")

if avg['average_correlation'] > 0.5:
    print(f"   Strong correlation between Google and predicted ChatGPT trends")
elif avg['average_correlation'] > 0.2:
    print(f"   📊 Moderate correlation between Google and predicted ChatGPT trends")
else:
    print(f"   📉 Weak correlation between Google and predicted ChatGPT trends")

print(f"\n Analysis completed successfully!")

# Trendline

In [ ]:
# Import required functions
from embeddingsKNN import promptToKeywordsAndAnalyze, loadTrainingData, createTrendlinePlot
import matplotlib.pyplot as plt
import numpy as np

# Load training data
print("🔄 Loading training data...")
loadTrainingData()
print("✅ Training data loaded successfully")

# Run inference on the prompt
print("\n" + "="*70)
print("🚀 RUNNING INFERENCE ON PROMPT: 'is the model 3 a good car?'")
print("="*70)

# Execute the analysis
result = promptToKeywordsAndAnalyze(
    prompt="is the model 3 a good car?",
    startDate="2025-06-01",
    endDate="2025-06-30",
    k=3,
    weights='distance'
)

print(f"\n Analysis completed!")
print(f"📝 Original prompt: '{result['prompt']}'")
print(f" Generated keywords: {result['generated_keywords']}")
print(f"✅ Successfully analyzed: {result['successful_keywords']}")
print(f" Keywords processed: {result['average_result']['num_keywords']}")

In [ ]:
# Explicitly create the trendline plot to ensure it shows
print("\n" + "="*70)
print("�� CREATING TRENDLINE PLOT")
print("="*70)

# Call the trendline function explicitly
createTrendlinePlot(
    individual_results=result['individual_results'],
    successful_keywords=result['successful_keywords'],
    prompt=result['prompt']
)

print("✅ Trendline plot created!")

In [ ]:
# Create an additional custom trendline visualization
print("\n" + "="*70)
print("📊 CUSTOM TRENDLINE ANALYSIS")
print("="*70)

# Extract time series data manually
individual_results = result['individual_results']
successful_keywords = result['successful_keywords']

if individual_results:
    # Get dates from first result
    first_result = individual_results[0]
    dates = [pd.to_datetime(item['date']) for item in first_result['google_trends']['data']]
    
    # Extract trends
    chatgpt_trends = []
    google_trends = []
    
    for result_item in individual_results:
        chatgpt_trends.append(result_item['predicted_chatgpt_trend']['trend_values'])
        google_trends.append(result_item['google_trends']['trend_values'])
    
    # Convert to numpy arrays
    chatgpt_trends = np.array(chatgpt_trends)
    google_trends = np.array(google_trends)
    
    # Calculate averages
    avg_chatgpt_trend = np.mean(chatgpt_trends, axis=0)
    avg_google_trend = np.mean(google_trends, axis=0)
    
    # Create custom trendline plot
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12))
    
    # Plot 1: Individual ChatGPT trends
    colors = ['#ff6b6b', '#4ecdc4', '#45b7d1', '#96ceb4', '#feca57']
    for i, (keyword, trend) in enumerate(zip(successful_keywords, chatgpt_trends)):
        color = colors[i % len(colors)]
        ax1.plot(dates, trend, color=color, alpha=0.7, linewidth=2, 
                label=f'{keyword}', linestyle='--')
    
    # Plot average trendline
    ax1.plot(dates, avg_chatgpt_trend, color='red', linewidth=4, 
            label='Average Predicted ChatGPT', linestyle='-')
    
    ax1.set_xlabel('Date', fontsize=12)
    ax1.set_ylabel('Predicted ChatGPT Trend Value', fontsize=12)
    ax1.set_title('Predicted ChatGPT Trends Over Time\nPrompt: "is the model 3 a good car?"', 
                 fontsize=16, fontweight='bold')
    ax1.legend(fontsize=10)
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim(0, 100)
    
    # Format x-axis
    ax1.xaxis.set_major_locator(plt.matplotlib.dates.DayLocator(interval=3))
    ax1.xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%m/%d'))
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)
    
    # Plot 2: Comparison with Google Trends
    ax2.plot(dates, avg_google_trend, color='blue', linewidth=3, 
            label='Average Google Trends', linestyle='-')
    ax2.plot(dates, avg_chatgpt_trend, color='red', linewidth=3, 
            label='Average Predicted ChatGPT', linestyle='--')
    
    ax2.set_xlabel('Date', fontsize=12)
    ax2.set_ylabel('Trend Value', fontsize=12)
    ax2.set_title('Google Trends vs Predicted ChatGPT Trends (Average)', 
                 fontsize=16, fontweight='bold')
    ax2.legend(fontsize=10)
    ax2.grid(True, alpha=0.3)
    ax2.set_ylim(0, 100)
    
    # Format x-axis
    ax2.xaxis.set_major_locator(plt.matplotlib.dates.DayLocator(interval=3))
    ax2.xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%m/%d'))
    plt.setp(ax2.xaxis.get_majorticklabels(), rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # Print trendline statistics
    print(f"\n{'='*60}")
    print("TRENDLINE ANALYSIS SUMMARY")
    print(f"{'='*60}")
    print(f"Time period: {dates[0].strftime('%Y-%m-%d')} to {dates[-1].strftime('%Y-%m-%d')}")
    print(f"Number of data points: {len(dates)}")
    print(f"Average ChatGPT trend range: {np.min(avg_chatgpt_trend):.1f} - {np.max(avg_chatgpt_trend):.1f}")
    print(f"Average Google trend range: {np.min(avg_google_trend):.1f} - {np.max(avg_google_trend):.1f}")
    
    # Calculate trend direction
    chatgpt_trend_direction = "↗️ Increasing" if avg_chatgpt_trend[-1] > avg_chatgpt_trend[0] else "↘️ Decreasing"
    google_trend_direction = "↗️ Increasing" if avg_google_trend[-1] > avg_google_trend[0] else "↘️ Decreasing"
    
    print(f"ChatGPT trend direction: {chatgpt_trend_direction}")
    print(f"Google trend direction: {google_trend_direction}")
    
    # Calculate volatility
    print(f"ChatGPT trend volatility: {np.std(avg_chatgpt_trend):.2f}")
    print(f"Google trend volatility: {np.std(avg_google_trend):.2f}")
    
else:
    print("❌ No individual results available for trendline analysis")

In [ ]:
# Display final summary
print("\n" + "="*70)
print("🎯 FINAL ANALYSIS SUMMARY")
print("="*70)

avg = result['average_result']
print(f"📝 Original Prompt: '{result['prompt']}'")
print(f" Generated Keywords: {', '.join(result['generated_keywords'])}")
print(f"✅ Successfully Analyzed: {', '.join(result['successful_keywords'])}")
print(f"📊 Analysis Period: June 2025")
print(f" Keywords: {avg['num_keywords']}")

print(f"\n📈 PERFORMANCE METRICS:")
print(f"   📊 Average Google Trend: {avg['average_google_trend']:.2f} ± {avg['std_google_trend']:.2f}")
print(f"   🤖 Average Predicted ChatGPT: {avg['average_predicted_chatgpt']:.2f} ± {avg['std_predicted_chatgpt']:.2f}")
print(f"   Average Confidence: {avg['average_confidence']:.3f} ± {avg['std_confidence']:.3f}")
print(f"   📊 Average Correlation: {avg['average_correlation']:.3f} ± {avg['std_correlation']:.3f}")

print(f"\n Analysis completed successfully!")
print(f"�� You should now see the trendline plots showing how ChatGPT trends evolve over time!")